# Docker Compose

## Docker Compose: la herramienta todo en uno

Todo lo que hemos hecho hasta ahora esta bien, pero es un poco tedioso. Para ello esta docker compose.

Si estas usando Windows esta instalada por defecto con docker desktop. Si estas usando Linux debes instalarla. Parece que ya la tengo. 

Docker Compose lo que nos permite es describir de forma declarativa la arquitectura de servicios que nuestra aplicacion necesita. Y se hace desde un pequeño archivo, para que docker corra casi todos los comandos que venimos ejecutando manualmente.

Vamos al proyecto en el **folder docker** donde hay un archivo llamado *docker-compose.yml*. Basicamente es una estructura declarativa.

![](https://i.imgur.com/rXZvpfz.png)

- services: indica los servicios que componen la aplicacion, en nuestro caso esta compuesta por dos servicios o microservicios: una que se llama **app** y otra que se llama **db** como en la seccion anterior.

- Donde se indica la imagen de mongo, le indico la version, pues ya tengo una imagen de dicha version.

Un servicio de compose es o viene siendo equivalente al concepto de container, donde **app** y **db** son dos contenedores. Aunque hay una sutil diferencia.

Porque un servicio, puede tener uno o mas contenedores de la misma imagen. Y tener por ejemplo mas contenedores de **app** para poder facilmente tener mas BW en nuestra aplicacion. 

- El statement *depends_on*, indicamos que dicho contenedor va a depender de el servicio *db*. Es decir, que **db** necesita ser creado antes que **app**

### Ejercicio.

Encontrar similitudes entre la figura anterior y los siguientes comandos usados anteriormente:

    docker run -d --name db mongo:5.0

    docker run -d --name app -p 3000:3000 --env MONGO_URL=mongodb://db:27017/test platziapp

### Aprendiendo a usar docker compose

#### Preparando el entorno

Empezemos por remover los contenedores previamente creados y evitar posibles conflictos:

![](https://i.imgur.com/39LyW8g.png)

    docker rm app db

Igualmente interesante, es lo descrito por el instructor, donde los procesos del ejercio anterior estan corriendo, y como al intentar crear un nuevo proceso usando los mismos puertos, pues va a generar un conflico

![](https://i.imgur.com/oMNxqh5.png)

#### Listo

Simplemente:

    docker compose up

Y empezara a correr y a llenar mi consola porque no lo corri en modo detach. Y empezara a mostrar los logs de ambas aplicaciones.

![](https://i.imgur.com/u9rCFJV.png)



    


- Crea los Containers, con un nombre propio, para evitar posibles conflictos con otros contenedores esten corriendo y tengan el mismo nombre. 

- Los containers empiezan por docker, no por docker, sino porque estoy en la carpeta docker 😏

- Fijate el orden crea los contenedores.

- A diferencia del instructor, no hay una evidencia haya creado una red

- y los atañe aparentemente a la misma red.

Si vamos al navegador en el *http://localhost:3000/* veremos esta corriendo 😎

Parece ser hay unas sutiles diferencias, de mis logs a los mostrados por el instructor. 

### Levantando el servicio en modo detach

    docker compose up -d

![](https://i.imgur.com/5QFMzye.png)

Y el resultado es el mismo:

![](https://i.imgur.com/ZkqFZLx.png)

#### Analizando 

![](https://i.imgur.com/HN9MRe8.png)

- app esta exponiendo el servicio en el puerto 3000 como era de esperarse y puedo accederlo desde mi maquina.

- pero db expone el puerto 27017 pero solo para la red interna.

¿Y cual es la red interna?


Inspeccionando cada una de las redes de docker encontre que:

    docker network inspect docker_default

![](https://i.imgur.com/Rz91spZ.png)

Los dos containers estan conectados en la red *docker default*



## Subcomandos de Docker Compose

👉 Importante aclarar que en el statement de la variable de entorno, en el compose file, **db** esta haciendo referencia al otro servicio, no necesito saber el nombre del contenedor(en perse) como fue fue el caso en la seccion de *docker network*

![](https://i.imgur.com/nS6sBTN.png)

### Ver los logs

Para ver lo de todos los servicios:

    docker compose logs 

Para ver solamente de un servicio:

    docker compose logs app

![](https://i.imgur.com/Y9nm16M.png)

Y para verlos en tiempo real, o a medida que se van generando:

    docker compose logs -f app

Y para el otro servicio obviamente:

    docker compose logs db

#### Corriendo un comando o proceso especifico en un contenedor

Muy facil, con exec, y en vez de colocar el nombre del contenedor, pues el nombre del servicio:

    docker compose exec app bash

Y efectivamente, vamos a poder ver nuestro codigo 

![](https://i.imgur.com/1WexGWK.png)






#### Borrando todo

    docker compose down 

E increiblemente, borra los contenedores tambien 😁

## Docker Compose como herramienta de desarrollo

### Remplazando image por build

supongamos realizamos unos cambios a nuestro codigo en el ultimo momento:

![](https://i.imgur.com/uCTsK2U.png)

Y para verlos reflejados tendriamos que buildiar la imagen, y hay si usar docker compose.

Sin embargo, *docker composer* nos permite un enfoque mas practico, modificando el *docker-compose.yml*:

![](https://i.imgur.com/xPzyNzl.png)

-  cambiamos la instruccion *image*, por *build*.
-  Como queremos darle un contexto a build que es el mismo directorio donde estamos trabajando, le colocamos el puntico.
- el servicio *db* se sigue asociando con una imagen.

Asegurarnos de estar en el folder del contexto de build, en mi caso *docker*, y:

    docker compose build

Tambien hubieramos podido usar, siendo especificos para que buildee un servicio especifico:

    docker compese build app

Y ahora si:

    docker compose up -d

No deberia tardar nada, y sin embargo, lo esta haciendo. Finalmente cuando lo hace verificamos cual fue la imagen que uso para buildiar app:


![](https://i.imgur.com/gbH08vt.png)

- anteriormente habia usado la imagen que teniamos en disco: *platziapp*, pero esta vez esta usanod la imagen que acabo de buildiar con *compose build*: **docker-app**

Pero tal vez, este enfoque no es el mas practico, porque queremos escribir codigo, y que esos cambios pasen directamente a la imagen o container que estamos trabajando, y no hacer un build cada vez.

### Volumes

A la altura del servico, agregamos una nueva instruccion, ojo con las indentaciones. Y hacemos un bind mount.

![](https://i.imgur.com/nxzfpxd.png)

- Que se monte lo que esta en mi ruta actual(.) en(:) usr/src

Y ahora si:

    docker compose up -d

Y todo deberia estar bien, pero no 😱

![](https://i.imgur.com/fQTCZvm.png)

#### Solucion

Algo se rompio, y no sabemos que fue:

- con *docker ps*, observamos solo esta corriendo el servico de mongo.
- investigando que paso con *docker compose logs app*

Dice que no puede encontrar un modulo, y esto paso en una clase anterior, y para solucionarlo hubo que montar solo *index.js*

Esto pasa porque se esta montando un directorio que no tiene instalado los modulos de node, y por tanto *node* no va poder arrancar porque le faltan esas dependencias. 

Con la siguiente instruccion le decimos que cuando hagas el *bind mount* ignore una ruta dentro del contenedor, que es donde esta los modulos cuando se hace el build. Y le estas diciendo que no montes nada encima de esa ruta.

![](https://i.imgur.com/ucaRzOE.png)

- fijate no lleva los dos puntos(:) ni nada

    docker compose up -d

Comprobamos que ambos servicioes esten corriendo, y vamos al navegador:

Y ahora si 😎

#### Verificando este tomando los cambios

hago un camio en mi codigo: *index.js* y miro si efectivamente esta tomando los cambios, y que crees que paso. NADA 😩😩

Si te acuerdas anteriormente, tuvimos que modificar el codigo para que *node*, y tambien en el *Dockerfile* para implementar que *nodedemon* monitoreara los cambios.

¿Habra una manera mas practica de hacerlo?

Agregamos una nueva instruccion:

![](https://i.imgur.com/CjMcbvT.png)

Si analizamos los logs de la aplicacion con:

    docker compose logs app

![](https://i.imgur.com/HlueM5C.png)

Ahora si esta funcionando como era su proposito 😏😏